In [ ]:
# doesnt save but optimized clean csv
# import pandas as pd

import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd

def load_articles():
    # Handle irregular CSV format
    df = pd.read_csv('articles.csv', 
                    sep='\t',  # Use tab separator if commas in content
                    on_bad_lines='skip',
                    header=None,
                    names=['articles'])
    
    # Filter valid articles
    return df[df['articles'].str.contains(r'\w{50,}', na=False)]  # Only keep real content

def clean_text(text):
    # Remove navigation/footer patterns
    patterns = [
        r'Drinks Food.*?Subscribe',
        r'Amaro Montenegro.*?MIA airport location',
        r'Create Next App',
        r'Follow us on Instagram\s+HERE',
        r'Check out their instagram.*?here'
    ]
    for pattern in patterns:
        text = re.sub(pattern, '', text, flags=re.DOTALL)
    return text.strip()

def process_articles():
    df = load_articles()
    df['clean_text'] = df['articles'].apply(clean_text)
    return [text for text in df['clean_text'].unique() if len(text) > 500]

# Process articles and save to CSV
articles = process_articles()
pd.DataFrame(articles, columns=['content']).to_csv('processed_articles.csv', index=False)

In [ ]:
# works - creates vector store

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import pandas as pd
import os

def create_optimized_vector_store(csv_path):
    # 1. Load and preprocess articles with proper CSV handling
    df = pd.read_csv(
        csv_path,
        quoting=1,  # QUOTE_ALL
        escapechar='\\',  # Handle escaped characters
        encoding='utf-8'
    )
    
    # Ensure the 'articles' column exists
    if 'articles' not in df.columns:
        raise ValueError("CSV file must contain an 'articles' column")
    
    articles = df['articles'].fillna('').tolist()
    
    # 2. Improved text splitting with context-aware chunking
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,  # Optimal for preserving paragraph context
        chunk_overlap=100,  # Maintain context between chunks
        separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""]  # Prioritize paragraph breaks
    )
    
    # 3. Create document objects with metadata and error handling
    documents = []
    for idx, article in enumerate(articles):
        try:
            # Basic cleaning
            cleaned_article = " ".join(str(article).strip().split())
            
            chunks = text_splitter.split_text(cleaned_article)
            for i, chunk in enumerate(chunks):
                documents.append(Document(
                    page_content=chunk,
                    metadata={"source": f"article_{idx}", "chunk": i}
                ))
        except Exception as e:
            print(f"Warning: Could not process article {idx}: {str(e)}")
    
    if not documents:
        raise ValueError("No valid documents were created from the input articles")
    
    # 4. Initialize embeddings with batch processing
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'mps'},
        encode_kwargs={
            'normalize_embeddings': True,
            'batch_size': 32  # Optimize for GPU/MPS performance
        }
    )
    
    # 5. Create FAISS index with optimized parameters
    vector_store = FAISS.from_documents(
        documents=documents,
        embedding=embeddings,
        distance_strategy="COSINE"  # Matches normalized embeddings
    )
    
    # 6. Save with verification
    save_path = os.path.abspath("blog_faiss")
    vector_store.save_local(save_path)
    print(f"Optimized FAISS index saved to: {save_path}")
    print(f"Processed {len(documents)} chunks from {len(articles)} articles")
    
    return vector_store

# Usage example
if __name__ == "__main__":
    try:
        vector_store = create_optimized_vector_store("articles2.csv")
        print("Vector store created successfully!")
    except Exception as e:
        print(f"Error creating vector store: {str(e)}")

/var/folders/49/6ydqkbq172ngzt6p49xfm6b00000gn/T/ipykernel_98193/665401798.py:52: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


README.md: 0.00B [00:00, ?B/s]

Optimized FAISS index saved to: /Users/adamaslan/code/dfl/blog_faiss
Processed 46 chunks from 15 articles
Vector store created successfully!


In [ ]:
# more metrics - better embeddings

!pip install MultiVectorRetriever from langchain.retrievers.multi_vector

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.retrievers import MultiVectorRetriever, EmbeddingRetriever
from langchain.vectorstores.utils import filter_metadata
import pandas as pd
import os
import time
import numpy as np

def create_optimized_vector_store(csv_path):
    # 1. Load and preprocess articles with proper CSV handling
    df = pd.read_csv(
        csv_path,
        quoting=1,  # QUOTE_ALL
        escapechar='\\',
        encoding='utf-8'
    )
    
    if 'articles' not in df.columns:
        raise ValueError("CSV file must contain an 'articles' column")
    
    articles = df['articles'].fillna('').tolist()
    
    # 2. Improved text splitting
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100,
        separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""]
    )
    
    # 3. Create documents with metadata
    documents = []
    for idx, article in enumerate(articles):
        try:
            cleaned_article = " ".join(str(article).strip().split())
            chunks = text_splitter.split_text(cleaned_article)
            for i, chunk in enumerate(chunks):
                documents.append(Document(
                    page_content=chunk,
                    metadata={"source": f"article_{idx}", "chunk": i}
                ))
        except Exception as e:
            print(f"Warning: Could not process article {idx}: {str(e)}")
    
    if not documents:
        raise ValueError("No valid documents were created from the input articles")
    
    # 4. Initialize embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'mps'},
        encode_kwargs={
            'normalize_embeddings': True,
            'batch_size': 32
        }
    )
    
    # 5. Create FAISS index
    vector_store = FAISS.from_documents(
        documents=documents,
        embedding=embeddings,
        distance_strategy="COSINE"
    )
    
    # 6. Save index
    save_path = os.path.abspath("blog_faiss")
    vector_store.save_local(save_path)
    print(f"Optimized FAISS index saved to: {save_path}")
    print(f"Processed {len(documents)} chunks from {len(articles)} articles")
    
    # 7. Add LangChain enhancements
    retriever = EmbeddingRetriever(vectorstore=vector_store)
    multi_vector_retriever = MultiVectorRetriever(vectorstore=vector_store)
    filtered_docs = filter_metadata(documents, {"chunk": 0})  # Example filter

    # 8. Metrics block
    print("\n📊 Embedding & Retrieval Metrics:")
    try:
        # Embedding density
        sample_embeddings = embeddings.embed_documents([doc.page_content for doc in documents[:100]])
        norms = [np.linalg.norm(vec) for vec in sample_embeddings]
        avg_norm = np.mean(norms)

        # Redundancy
        unique_chunks = set(doc.page_content for doc in documents)
        redundancy_ratio = 1 - len(unique_chunks) / len(documents)

        # Retrieval latency
        start = time.time()
        _ = retriever.get_relevant_documents("sample query")
        latency = time.time() - start

        print(f"- Total chunks: {len(documents)}")
        print(f"- Unique chunks: {len(unique_chunks)}")
        print(f"- Redundancy ratio: {redundancy_ratio:.2f}")
        print(f"- Avg embedding norm: {avg_norm:.4f}")
        print(f"- Retrieval latency (sample query): {latency:.4f}s")
        print(f"- Metadata-filtered docs (chunk=0): {len(filtered_docs)}")
        print(f"- Coverage score: {len(documents) / len(articles):.2f} chunks/article")
        print(f"- Semantic cohesion (adjacent chunk similarity): TBD")
        print(f"- Recall@k / Precision@k: TBD (requires labeled queries)")
        print(f"- Query drift: TBD (requires query-to-centroid comparison)")
    except Exception as e:
        print(f"Metric computation error: {str(e)}")
    
    return vector_store

# Usage example
if __name__ == "__main__":
    try:
        vector_store = create_optimized_vector_store("articles2.csv")
        print("Vector store created successfully!")
    except Exception as e:
        print(f"Error creating vector store: {str(e)}")


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement MultiVectorRetriever (from versions: none)
ERROR: No matching distribution found for MultiVectorRetriever


ImportError: cannot import name 'filter_metadata' from 'langchain.vectorstores.utils' (/opt/homebrew/Caskroom/miniforge/base/envs/nunu24/lib/python3.12/site-packages/langchain/vectorstores/utils.py)

In [1]:
# with json - saves 
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Load the JSON file
json_path = "articles2.json"
with open(json_path, "r") as file:
    data = json.load(file)

# Filter out empty strings and extract valid content
texts = [value for key, value in data.items() if value.strip()]

# Initialize the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings
print("Generating embeddings...")
embeddings = model.encode(texts, convert_to_numpy=True)
print(f"Generated Embeddings Shape: {embeddings.shape}")

# Create a FAISS index with IDs
vector_dim = embeddings.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(vector_dim)  # L2 distance (Euclidean)
index_with_ids = faiss.IndexIDMap(index)

# Assign unique IDs based on the JSON keys
ids = np.array([int(key) for key in data.keys() if data[key].strip()], dtype=np.int64)
index_with_ids.add_with_ids(embeddings, ids)

# Save the FAISS index to a file
faiss.write_index(index_with_ids, "optimized_vector_database.faiss")
print("Optimized FAISS index saved as 'optimized_vector_database.faiss'.")

# Verify index size
print(f"FAISS index contains {index_with_ids.ntotal} entries.")

# Define the search function
def search(query, top_k=5):
    try:
        # Encode the query
        query_embedding = model.encode([query], convert_to_numpy=True)
        # Perform search in FAISS
        distances, results = index_with_ids.search(query_embedding, top_k)
        # Return results with distances
        return [(texts[i], distances[0][j]) for j, i in enumerate(results[0])]
    except Exception as e:
        print(f"An error occurred during search: {e}")
        return []

# Example search
query = "delicious Turkish dishes"

try:
    print("\nPerforming similarity search...")
    search_results = search(query, top_k=5)

    print("\nSimilarity Search Results:")
    for result, distance in search_results:
        print(f"Text: {result}\nDistance: {distance}\n")
except Exception as e:
    print(f"Error while performing search: {e}")

Generating embeddings...
Generated Embeddings Shape: (100, 384)
Optimized FAISS index saved as 'optimized_vector_database.faiss'.
FAISS index contains 100 entries.

Performing similarity search...


: 

In [ ]:
# claude - make faiss
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from tqdm import tqdm  # For progress tracking
import gc  # For garbage collection

def process_in_batches(json_path, batch_size=100):
    # Initialize model
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    vector_dim = model.get_sentence_embedding_dimension()
    
    # Initialize FAISS index
    index = faiss.IndexFlatL2(vector_dim)
    index_with_ids = faiss.IndexIDMap(index)
    
    # Process in batches
    batch_texts = []
    batch_ids = []
    id_to_text = {}  # Store mapping of ID to text
    current_id = 0
    
    print("Processing data in batches...")
    with open(json_path, "r") as file:
        data = json.load(file)
        
        for key, value in tqdm(data.items()):
            if value.strip():  # Skip empty strings
                batch_texts.append(value)
                batch_ids.append(current_id)
                id_to_text[current_id] = value
                current_id += 1
                
                if len(batch_texts) >= batch_size:
                    # Generate embeddings for batch
                    embeddings = model.encode(batch_texts, convert_to_numpy=True)
                    # Add to index
                    index_with_ids.add_with_ids(embeddings, np.array(batch_ids, dtype=np.int64))
                    
                    # Clear batch and garbage collect
                    batch_texts = []
                    batch_ids = []
                    gc.collect()
    
    # Process remaining items
    if batch_texts:
        embeddings = model.encode(batch_texts, convert_to_numpy=True)
        index_with_ids.add_with_ids(embeddings, np.array(batch_ids, dtype=np.int64))
    
    # Save the index and mapping
    faiss.write_index(index_with_ids, "optimized_vector_database.faiss")
    with open("id_to_text_mapping.json", "w") as f:
        json.dump(id_to_text, f)
    
    return index_with_ids, id_to_text

def search(query, index_with_ids, id_to_text, model, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, result_ids = index_with_ids.search(query_embedding, top_k)
    
    return [(id_to_text.get(str(id_), ""), distances[0][i]) 
            for i, id_ in enumerate(result_ids[0])]

# Usage
try:
    json_path = "articles2.json"
    index_with_ids, id_to_text = process_in_batches(json_path)
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    
    query = "delicious Turkish dishes"
    results = search(query, index_with_ids, id_to_text, model)
    
    for text, distance in results:
        print(f"Distance: {distance}\nText: {text}\n")

except Exception as e:
    print(f"Error: {e}")

: 

In [1]:
# vector b

import faiss
import numpy as np

# Load the FAISS index
index = faiss.read_index("articles2.faiss")

# Generate some random query vectors (use real data if available)
query_vectors = np.random.random((5, index.d)).astype("float32")  # 5 queries

# Perform a search on the index
k = 10  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_vectors, k)

# Display results
print("Query Results:")
for i, (d, idx) in enumerate(zip(distances, indices)):
    print(f"Query {i+1}:")
    print("  Nearest neighbor distances:", d)
    print("  Nearest neighbor indices:", idx)


: 

In [8]:
# saves now but didnt show path before
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import pickle

def create_vector_store(chunks):
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'mps'},
        encode_kwargs={'normalize_embeddings': True}
    )
    
    vector_store = FAISS.from_documents(
        documents=chunks,
        embedding=embeddings
    )
    
    # # Save optimized index
    # vector_store.save_local("blog_faiss")
    # return vector_store

  # Save optimized index
    save_path = os.path.abspath("blog_faiss")  # Get absolute path
    vector_store.save_local(save_path)
    print(f"FAISS vector store saved to: {save_path}")
    return vector_store

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pickle

def split_text_into_chunks(articles, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap
    )
    chunks = []
    for article in articles:
        chunks.extend(text_splitter.split_text(article))
    return chunks

def create_vector_store(chunks):
    # Initialize embeddings model
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'mps'},  # Use Metal Performance Shaders (MPS) for macOS
        encode_kwargs={'normalize_embeddings': True}
    )
    
    # Create FAISS vector store
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    
    # Save FAISS index to project root
    vector_store.save_local("blog_faiss")
    print("FAISS vector store saved to 'blog_faiss'")
    return vector_store

# Usage:
# Assuming 'articles' is a list of cleaned article content
articles = process_articles()  # Use your existing `process_articles()` function
chunks = split_text_into_chunks(articles)
vector_store = create_vector_store(chunks)

IndexError: list index out of range

# Cell 2: Initialize components
from typing import List
from ollama import Client
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'mps'}  # Use Apple Metal
)

# Cell 3: RAG Class
class BlogRAG:
    def __init__(self):
        try:
            self.client = Client(host='http://localhost:11434')
            self.retriever = FAISS.load_local(
                "blog_faiss",
                embeddings
            ).as_retriever(search_kwargs={"k": 3})
        except Exception as e:
            raise RuntimeError(f"Failed to initialize RAG system: {str(e)}")

    def format_prompt(self, context: List[str], question: str) -> str:
        return f"""Answer using ONLY these blog excerpts:
        {"".join([f'• {doc.page_content}\n' for doc in context])}
        
        Question: {question}
        Answer in the blog's casual food writing style using under 200 words:"""

    def ask(self, question: str) -> str:
        try:
            context = self.retriever.get_relevant_documents(question)
            prompt = self.format_prompt(context, question)
            
            response = self.client.generate(
                model='phi3',
                prompt=prompt,
                options={'temperature': 0.3, 'num_ctx': 2048}
            )
            return response['response']
        except Exception as e:
            return f"Error generating answer: {str(e)}"

# Cell 4: Test the system
if __name__ == "__main__":
    rag = BlogRAG()
    sample_question = "Where can I get good Turkish food in New York?"
    print(f"Question: {sample_question}")
    print("Answer:", rag.ask(sample_question))

In [5]:
# Cell 1: Install requirements
%pip install langchain sentence-transformers faiss-cpu transformers torch

# Cell 2: Initialize components
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
from typing import List

# Initialize embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},  # Use 'mps' for Apple Silicon
    encode_kwargs={'normalize_embeddings': True}
)

# Initialize local LLM (using a smaller public model)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.3
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Cell 3: RAG Class
class BlogRAG:
    def __init__(self):
        try:
            self.vector_store = FAISS.load_local(
                "blog_faiss",
                embeddings,
                allow_dangerous_deserialization=True
            )
            self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 3})
        except Exception as e:
            raise RuntimeError(f"Vector store load failed: {str(e)}")

    def format_prompt(self, context: List[str], question: str) -> str:
        return f"""Based on these food blog excerpts:
        {" ".join([doc.page_content[:500] for doc in context])}
        
        Answer this question: {question}
        Keep the answer under 150 words and casual:"""
    
    def ask(self, question: str) -> str:
        try:
            context = self.retriever.get_relevant_documents(question)
            prompt = self.format_prompt(context, question)
            
            response = llm(
                prompt,
                max_new_tokens=200,
                do_sample=True
            )
            return response[0]['generated_text'].split("casual:")[-1].strip()
        except Exception as e:
            return f"Error: {str(e)}"

# Cell 4: Test
if __name__ == "__main__":
    rag = BlogRAG()
    sample_question = "Where can I find good Turkish food in NYC?"
    print(f"Question: {sample_question}")
    print("Answer:", rag.ask(sample_question))

python(17065) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/2b/64/2b642798915fc368e7b638986f68446b121c1d59b30075e146bd6312ee664ac2/6e6001da2106d4757498752a021df6c2bdc332c650aae4bae6b0c004dcf14933?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1738037249&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODAzNzI0OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzJiLzY0LzJiNjQyNzk4OTE1ZmMzNjhlN2I2Mzg5ODZmNjg0NDZiMTIxYzFkNTliMzAwNzVlMTQ2YmQ2MzEyZWU2NjRhYzIvNmU2MDAxZGEyMTA2ZDQ3NTc0OTg3NTJhMDIxZGY2YzJiZGMzMzJjNjUwYWFlNGJhZTZiMGMwMDRkY2YxNDkzMz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=IljEF7u50InTXhQKUxf%7EQBC7I2DAtZwyf2wQtdMriXaSSJFJGjfssqwsw05xfQabR6hPJ7pfzuZ1jMUAPIGVfj7OR-I6Tz81rT2jd8jlWOtvwBH-yQ0szNXn7ukOylx3ryxJw3%7EKhTAJyCqcIu1A%7EtdQx6EkJK-l2UlNMm0kiv8GRsIgqUBU03zImj5Q5rICEZWLVoyijQu2S9PKiSd5A0Qz6%7ECtPdZEOEjdZgudXGbRweCI8DTFO91Vftvy

model.safetensors:  45%|####5     | 996M/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the disk.
/var/folders/49/6ydqkbq172ngzt6p49xfm6b00000gn/T/ipykernel_11819/1148055367.py:35: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


RuntimeError: Vector store load failed: Error in faiss::FileIOReader::FileIOReader(const char *) at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/impl/io.cpp:68: Error: 'f' failed: could not open blog_faiss/index.faiss for reading: No such file or directory